# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from utils.ml import *
import winsound

# Action Points

## 1. No need to make 10 transactions a day. Just one is enough
## 2. See the lema gap size on the losing months
## 3. Can I use actual tick value (close) instead of lema for moves?

data                = get_date_list(data)
data                = read_data(data)

In [2]:
%%time
data = run_engine(data)

100%|██████████████████████████████████| 74387/74387 [00:23<00:00, 3136.94it/s]


pl    0.08381
dtype: float64
--------------------------------------
            DateTime_frmt  order_side  close_type       pl
70027 2020-12-10 04:05:00         NaN  sema_close -0.00165
70178 2020-12-10 16:40:00         NaN  sema_close -0.00152
70405 2020-12-11 11:35:00         NaN  sema_close -0.00117
70533 2020-12-13 22:15:00         NaN  sema_close -0.00265
70841 2020-12-14 23:55:00         NaN  sema_close -0.00052
70934 2020-12-15 07:40:00         NaN  sema_close -0.00118
71751 2020-12-18 03:45:00         NaN  sema_close  0.00903
72229 2020-12-21 19:35:00         NaN  sema_close -0.00118
72442 2020-12-22 13:20:00         NaN  sema_close -0.00053
72662 2020-12-23 07:40:00         NaN  sema_close -0.00246
73033 2020-12-24 14:35:00         NaN  sema_close -0.00131
73121 2020-12-24 21:55:00         NaN  sema_close -0.00045
73173 2020-12-25 02:45:00         NaN  sema_close -0.00079
73268 2020-12-28 00:45:00         NaN  sema_close -0.00010
74250 2020-12-31 10:35:00         NaN  sema_clo

100%|██████████████████████████████████████| 380/380 [00:00<00:00, 1115.73it/s]

Wall time: 2min 6s


In [3]:
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')
monthly_avg = np.round(data['df_ohlc']['pl'].sum()/len(set(data['df_ohlc']['month_val'])),4)
print(f'monthly_avg : {monthly_avg}')

month_val
1     0.00376
2     0.01776
3     0.02985
4    -0.00228
5     0.02848
6     0.00002
7    -0.00399
8    -0.01319
9     0.00694
10   -0.00389
11    0.00897
12    0.01138
Name: pl, dtype: float64
---------------------
max : 3 : 0.029849999999999998
min : 8 : -0.01319
---------------------
monthly_avg : 0.007


In [4]:
data['lema_gap_pl_analysis_month'] = 8

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

,index,positive,negative,count_diff,positive_sum,negative_sum,sum_diff
2,0.000,0.0,2.0,-2.0,0.00000,-0.00586,-0.00586
0,0.001,1.0,4.0,-3.0,0.00270,-0.00706,-0.00436
1,0.002,2.0,4.0,-2.0,0.00269,-0.00929,-0.00660
6,0.003,1.0,0.0,1.0,0.00283,0.00000,0.00283
3,0.004,1.0,1.0,0.0,0.00629,-0.00249,0.00380
5,0.005,1.0,0.0,1.0,0.00149,0.00000,0.00149
4,0.006,0.0,1.0,-1.0,0.00000,-0.00179,-0.00179


In [5]:
data['lema_gap_pl_analysis_month'] = 3

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

,index,positive,negative,count_diff,positive_sum,negative_sum,sum_diff
0,0.001,2.0,3.0,-1.0,0.00789,-0.00859,-0.00070
3,0.002,1.0,2.0,-1.0,0.01968,-0.00246,0.01722
1,0.003,0.0,3.0,-3.0,0.00000,-0.00724,-0.00724
7,0.004,1.0,0.0,1.0,0.00581,0.00000,0.00581
6,0.007,2.0,0.0,2.0,0.01588,0.00000,0.01588
2,0.010,0.0,2.0,-2.0,0.00000,-0.00286,-0.00286
4,0.011,1.0,1.0,0.0,0.00692,-0.00126,0.00566
5,0.012,0.0,1.0,-1.0,0.00000,-0.00392,-0.00392


In [6]:
y = data['df_ohlc']['pl'].sum()
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
send_telegram_message(f'{data["input_year"]}')
send_telegram_message(f'{y}')
send_telegram_message(f'{x}')
send_telegram_message(f'-------------------')

In [7]:
sys.exit()

SystemExit: 

In [ ]:
y = data['df_ohlc'][data['df_ohlc']['month_val'] == 6]
x = y.groupby(['date_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

In [ ]:
data['df_ohlc'].tail()

In [ ]:
data['df_ohlc']['lema_gap'].describe()

In [ ]:
x = 5
data['df_ohlc'].iloc[-5:,x:x+15]

In [ ]:
data['df_small']['lema_gap'].describe()

In [ ]:
abs(all['negative_sum']) / all['positive_sum']

In [ ]:
df_small['order_side']

In [ ]:
data['df_small'].to_csv('data/temp.csv')

In [8]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
month = 2

# Monthly Chart -------------------
# data['plot_start']  = f"{data['input_year']}-{month}-1"
# data['plot_stop']   = f"{data['input_year']}-{month+1}-1"
# Monthly Chart -------------------

# December Chart ----------------------------
# data['plot_stop']   = f"{data['input_year']+1}-1-1"
# December Chart ----------------------------

# Year Chart ----------------------------
data['plot_start']  = f"{data['input_year']}-1-1"
data['plot_stop']   = f"{data['input_year']+1}-1-1"
# Year Chart ----------------------------

# data['things_to_plot'] = ['tick', 'ema', 'BBands', 'dots']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'indicators']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'cross']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'tip']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'lema_angle']
# data['things_to_plot'] = ['ema', 'sema_angle', 'slema_angle', 'lema_angle', 'dir']
# data['things_to_plot'] = ['ema', 'positions', 'lema_angle','slema_angle','sema_angle', 'rsi']
# data['things_to_plot'] = ['ema','positions', 'lema_angle','lema_diff', 'lema_max_min']
data['things_to_plot'] = ['sema', 'lema', 'lema_angle']

plot_graph(data)

generate_result_report(data)